**Exercise 7.1**

In Chapter 6 we noted that the Monte Carlo error can be written as the sum of TD errors (6.6) if the value estimates don't change from step to step. Show that the n-step error used in (7.2) can also be written as a sum of TD errors (again if the vavlue estimates don't cahnge) generalizing the earlier result.


Reminder of equation (7.2)

\begin{aligned}
    V_{t+n}(S_t) = V_{t+n-1}(S_t) + \alpha [ G_{t:t+n} - V_{t+n-1}(S_t) ]
\end{aligned}

**ANSWER**

\begin{aligned}
    G_t - V(S_t) &= R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \cdots \gamma^{T-t-1} R_T - V(S_t) + V(S_{t+n}) - V(S_{t+n}) \\
    &= \delta_t + \gamma^n (G_{t+n} - V(S_{t+n})) \\
    &= \delta_t + \gamma^n \delta_{t+n} + \gamma^2n (G_{t+2n} - V(S_{t+2n})) \\
    &= \cdots \\
    &= \sum_{k=0}^{\bar{k}} \gamma^{kn} \delta_{t+kn}
\end{aligned}

where $\bar{k} = floor({\frac{T-t-1}{n}})$ and $\delta_t = G_{t:t+n} - V(S_t)$. (For $t+n > T$, $G_{t:t+n}$ is simply $G_t$)

For example, if $T=10, t=5, n=2, \bar{k}=floor(\frac{4}{2})=2$, then:

\begin{aligned}
    G_5 - V(S_5) = \delta_5 + \gamma^2 \delta_7 + \gamma^4 \delta_9
\end{aligned}

**Exercise 7.2 (programming)**

With an n-step method, the value estimates do change from step to step, so an algorithm that used the sum of TD errors (see previous exercise) in place of the error in (7.2) would actually be a shlightly different algorithm. Would it be a better algorithm or a worse one? Devise and program a small experiment to answer this question empirically.

**ANSWER**

I'm not sure I understand the question well. Using the sum of TD errors seems like a big change, not a slight change because we have to wait until the end of the episode to update our value estimates. I think it would probably be worse because we assumed $V$ doesn't change to write our MC error as a sum of TD errors.

**Exercise 7.3**

Why do you think a larger random walk task (19 states instead of 5) was used in the examples of this chapter? Would a smaller walk have shifted the avantage to a different value of n? How about the change in left-side outcome from 0 to -1 made in the larger walk? Do you think that made any difference in the best value of n?

**ANSWER**

- First, it allows us to see the results for larger n
- Second, it reduces the variance in value estimates. Consider the following setting $\alpha=\frac{1}{2}, n=5, V(S_t)=0.5$ with 5 non-terminal states. Imagine that in the first episode, the agent finds itself at state A, then the agent only moves right 5 times in a row. It could happen with probability $(\frac{1}{2})^5 = \frac{1}{32} \approx 3\% $. It would then update all the states A,B,C,D,E values to 0.75 exactly. (Because $V(S) \leftarrow 0.5 + \frac{1}{2} [1 - 0.5] = 0.75$ for all states). This is obviously not good for states A,B,C, whose real values are $\frac{1}{6}, \frac{2}{6}, \frac{3}{6}$. <br> 
Now with mores states, say n states, the probability that agent moves only right n times in a row starting from state A is more and more unlikely as n grows. With n=19, the probability is only $\frac{1}{2^{19}}$. High variance would not help us determining which n is better.

As for changing the left-side outcome from 0 to -1, I don't think it changes anything as long as the initial state values is set to 0. Consider the following setting $\alpha=\frac{1}{2}, n=2, V(S_t)=0$ with 5 non-terminal states:

Imagine that the agent goes left only, for a reward of +1 on the right side outcome only and 0.5 as initial value estimates, it would therefore do the following updates:
- $V(B) \leftarrow 0.5 + \frac{1}{2} [0 - 0.5] = 0.25$. **error**: $|0.25 - \frac{2}{6}| = \frac{1}{12}$
- $V(A) \leftarrow 0.5 + \frac{1}{2} [0 - 0.5] = 0.25$. **error**: $|0.25 - \frac{1}{6}| = \frac{1}{12}$
- 0.25 is the middle between the reward on left-side and the initial value, that is 0 and 0.5 

For a reward of +1 on the right side outcome, -1 on the left side, and 0 as initial value estimates:
- $V(B) \leftarrow 0 + \frac{1}{2} [-1 - 0] = -0.5$. **error**: $|-0.5 - (-\frac{2}{3})| = \frac{1}{6}$
- $V(A) \leftarrow 0 + \frac{1}{2} [-1 - 0] = -0.5$ **error**: $|-0.5 - (-\frac{1}{3})| = \frac{1}{6}$
- -0.5 is the middle between the reward on left-side and the initial value, that is -1 and 0

Except the change in the scale of the values and errors (and potentially the need to change the scale of $\alpha$), it doesn't fundamentally change the problem.

**Exercise 7.4**

Prove that the n-step return of Sarsa (7.4) can be written exactly in terms of a novel TD error, as:

\begin{align}
    G_{t:t+n} = Q_{t-1}(S_t,A_t) + \sum_{k=t}^{min(t+n,T)-1} \gamma^{k-t} [ R_{k+1} + \gamma Q_k(S_{k+1},A_{k+1}) - Q_{k-1}(S_k,A_k) ]
\end{align}

**ANSWER**

We can simply notice that in the sum, $\gamma Q_k(S_{k+1},A_{k+1})$ cancels $- Q_{k-1}(S_k,A_k)$ for the next k. It is a the sum of a telescoping series. The result is simply the first $- Q_{k-1}(S_k,A_k)$ and the last $\gamma Q_k(S_{k+1},A_{k+1})$ (ignoring $R_{k+1}$):

\begin{aligned}
    & \sum_{k=t}^{min(t+n,T)-1} \gamma^{k-t} [ R_{k+1} + \gamma Q_k(S_{k+1},A_{k+1}) - Q_{k-1}(S_k,A_k) ] \\
    &= 
    \begin{cases}
        R_{t+1} - Q_{t-1}(S_t,A_t) + \gamma^{n-1} R_{t+n} + \gamma^n Q_{t+n-1}(S_{t+n},A_{t+n}) & \text{if } t+n<T \\
        R_{t+1} - Q_{t-1}(S_t,A_t) + \gamma^{T-1} R_{T} & \text{otherwise}
    \end{cases}
\end{aligned}

Therefore
\begin{aligned}
    & Q_{t-1}(S_t,A_t) + \sum_{k=t}^{min(t+n,T)-1} \gamma^{k-t} [ R_{k+1} + \gamma Q_k(S_{k+1},A_{k+1}) - Q_{k-1}(S_k,A_k) ] \\
    &= 
    \begin{cases}
        R_{t+1} + \gamma^{n-1} R_{t+n} + \gamma^n Q_{t+n-1}(S_{t+n},A_{t+n}) & \text{if } t+n<T \\
        R_{t+1} + \gamma^{T-1} R_{T} & \text{otherwise}
    \end{cases} \\
    &= G_{t:t+n}
\end{aligned}